In [ ]:
# TODO: update to the new data format, which simplifies things!
# TODO: update this from notebook to semi-interactive script

import csv
import re

def story_text_cleaned(text):
    if text:  # Check if the text is not None or NaN
        match = re.search(r"---\s*(.*?)\s*---", text, re.S)
        if match:
            return match.group(1).strip()
    text = text.replace("child", "").replace("son", "").replace("daughter", "").replace("boy", "").replace("girl", "")
    return text  # Return the original text if no match is found


# data = list(csv.DictReader(open('data/exp2-all_raw_data.csv', 'r')))
# race_of_parent
# data = list(csv.DictReader(open('data/exp3.csv', 'r')))
# religion_of_parent
data = list(csv.DictReader(open('data/exp4.csv', 'r')))

for line in data:
    line["story_text_cleaned"] = story_text_cleaned(line["story_text"])

for line in data:
    if "nationality_parent" not in line:
        continue
    x = line["nationality_parent"]
    line["nationality_parent_group"] = (
        "North A." if x in {"American", "Canadian"} else
        "South A." if x in {"Mexican", "Brazilian"} else
        "Euro." if x in {"British", "German", "French", "Italian", "Russian"} else
        "M. East." if x in {"Armenian", "Afghan", "Azerbaijani", "Egyptian", "Iranian", "Iraqi"} else
        "Africa" if x in {"Ethiopian", "Kenyan", "Malian", "Nigerian", "South African", "Sudanese"} else
        "Asia" if x in {"Chinese", "Filipino", "Indian", "Indonesian", "Japanese", "Sri Lankan", "Tajik", "Thai", "Vietnamese"} else
        "Other"
    )
    line["nationality_parent_developed"] = (
        "Developed" if x in {"American", "British", "Canadian", "French", "German", "Italian", "Japanese", "Russian"} else
        "Developing" if x in {"Afghan", "Armenian", "Azerbaijani", "Brazilian", "Chinese", "Egyptian", "Ethiopian", "Filipino", "Indian", "Indonesian", "Iranian", "Iraqi", "Kenyan", "Malian", "Mexican", "Nigerian", "South African", "Sri Lankan", "Sudanese", "Tajik", "Thai", "Vietnamese"} else
        "Other"
    )

In [ ]:
import numpy as np
import sklearn
import sklearn.neural_network
import sklearn.model_selection
import sklearn.feature_extraction.text
import sklearn.dummy

model = sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=100)

def predict_variable(
        in_variable="story_text_cleaned",
        out_variable="gender_of_child",
        filter_fn=lambda x: True,
    ):
    data_local = [line for line in data if filter_fn(line)]
    data_x = sklearn.feature_extraction.text.TfidfVectorizer(
        max_df=500,
        stop_words="english"
    ).fit_transform([line[in_variable] for line in data_local])
    data_y = [line[out_variable] for line in data_local]
    out = sklearn.model_selection.cross_validate(
        model,
        data_x,
        data_y,
        cv=5,
    )
    score = np.average(out["test_score"])
    print(f"{score:.1%}")

def predict_variable_dummy(out_variable="gender_of_child", filter_fn= lambda x: True):
    model = sklearn.dummy.DummyClassifier(strategy="most_frequent")
    data_local = [line for line in data if filter_fn(line)]
    data_x = [line for line in data_local]
    data_y = [line[out_variable] for line in data_local]
    out = sklearn.model_selection.cross_validate(
        model,
        data_x,
        data_y,
        cv=5,
    )
    score = np.average(out["test_score"])
    print(f"{score:.1%}")

In [ ]:
predict_variable_dummy(out_variable="gender_of_child")
predict_variable(in_variable="story_text_cleaned", out_variable="gender_of_child")
predict_variable(in_variable="story_text_cleaned", out_variable="gender_of_child", filter_fn=lambda x: x["model"] == "GPT4")
predict_variable(in_variable="story_text_cleaned", out_variable="gender_of_child", filter_fn=lambda x: x["model"] == "Llama3")

In [ ]:
predict_variable_dummy(out_variable="nationality_parent_developed")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_developed")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_developed", filter_fn=lambda x: x["model"] == "GPT4")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_developed", filter_fn=lambda x: x["model"] == "Llama3")

In [ ]:
predict_variable_dummy(out_variable="nationality_parent_group")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_group")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_group", filter_fn=lambda x: x["model"] == "GPT4")
predict_variable(in_variable="story_text_cleaned", out_variable="nationality_parent_group", filter_fn=lambda x: x["model"] == "Llama3")

In [ ]:
# NOTE: reload exp3
predict_variable_dummy(out_variable="race_of_parent")
predict_variable(in_variable="story_text_cleaned", out_variable="race_of_parent")
predict_variable(in_variable="story_text_cleaned", out_variable="race_of_parent", filter_fn=lambda x: x["model"] == "GPT4")
predict_variable(in_variable="story_text_cleaned", out_variable="race_of_parent", filter_fn=lambda x: x["model"] == "Llama3")

In [ ]:
# NOTE: reload exp4
predict_variable_dummy(out_variable="religion_of_parent")
predict_variable(in_variable="story_text_cleaned", out_variable="religion_of_parent")
predict_variable(in_variable="story_text_cleaned", out_variable="religion_of_parent", filter_fn=lambda x: x["model"] == "GPT4")
predict_variable(in_variable="story_text_cleaned", out_variable="religion_of_parent", filter_fn=lambda x: x["model"] == "Llama3")